In [202]:
import pandas as pd
from functions import count_by_time

In [203]:
df = pd.read_csv('yandex_music_project.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [204]:
df = df.rename(columns={
    '  userID': 'user_id',
    'Track': 'track',
    '  City  ': 'city',
    'Day': 'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [205]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [206]:
genre_df = df.dropna(subset=['genre'])
genre_df = genre_df.fillna('unknown')
df = df.fillna('unknown')
print(df.isna().sum(), genre_df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64 user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


In [207]:
df.duplicated().sum()

3826

In [208]:
genre_df.duplicated().sum()

3755

In [209]:
df = df.drop_duplicates().reset_index(drop=True)
genre_df = genre_df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum(), genre_df.duplicated().sum())

0 0


In [210]:
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


In [211]:
df = df.replace({"türk": "türkçe",
                "soul": "soulful",
                "new": "newage",
                "latin": "latino",
                "hip-hop": "hiphop",
                 "hip": "hiphop",
                "frankreich": "französisch",
                "africa": "afrikaans",
                "электроника": "electronic"})
genre_df = genre_df.replace({"türk": "türkçe",
                "soul": "soulful",
                "new": "newage",
                "latin": "latino",
                "hip-hop": "hiphop",
                "hip": "hiphop",
                "frankreich": "französisch",
                "africa": "afrikaans",
                "электроника": "electronic"})

Закончили предобработку данных:
1. Переименовали столбцы, чтобы с ними было удобней работать
2. Удалили все явные дубликаты
3. Убрали все неявные дубликаты

Проверим первую гипотезу.
Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
Для разных гипотез я сделал 2 разных DataFrame. Для первой гипотезы нам не важны жанры, а только активность, поэтому нам нужно больше данных. 
Для двух других гипотез, я удалил строки с неизвестными жанрами.

In [212]:
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

In [213]:
moscow_activity = df[df['city'] == 'Moscow']
md = moscow_activity.groupby('day')['user_id'].count()
md

day
Friday       15945
Monday       15740
Wednesday    11056
Name: user_id, dtype: int64

In [214]:
spb_activity = df[df['city'] == 'Saint-Petersburg']
sd = spb_activity.groupby('day')['user_id'].count()
sd

day
Friday       5895
Monday       5614
Wednesday    7003
Name: user_id, dtype: int64

In [215]:
data = [['Moscow', md[1], md[2], md[0]],
        ['Saint-Petersburg', sd[1], sd[2], sd[0]]]
compresion = pd.DataFrame(columns=['city', 'Monday', 'Wednesday', 'Friday'], 
                          data = data)
display(compresion)

,city,Monday,Wednesday,Friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


Как видно из этой таблицы, в Москве музыку слушали гораздо больше, чем в Санкт-Петербурге. Так что гипотеза подтвердилась. Тут еще можно заметить интересную особенность. В Санкт-Петербурге в среду виден пик слушателей, тогда как в Москве мы наблюдаем два пика в Понедельник и Пятницу. Чтобы как-то лучше проанализировать это нам нужны данные на всю неделю.

Проверим вторую гипотезу.
Утром в понедельник в Москве преобладают одни жанры музыки, а в Петербурге — другие. Это верно и для вечера пятницы.

За утро мы будем считать промежуток с 8:00 до 11:00.
За вечер мы будем считать промежуток с 17:00 до 20:00. 

In [216]:
moscow_genres = genre_df[genre_df['city'] == 'Moscow']
spb_genres = genre_df[genre_df['city'] == 'Saint-Petersburg']

In [217]:
count_by_time('Moscow', 'Monday', '08:00', '12:00', moscow_genres)

genre
pop           781
dance         549
electronic    480
rock          474
hiphop        285
             ... 
orchestral      1
podcasts        1
retro           1
rhythm          1
worldbeat       1
Name: genre, Length: 152, dtype: int64

In [218]:
count_by_time('Saint-Petersburg', 'Monday', '08:00', '12:00', spb_genres)

genre
pop           218
dance         182
rock          162
electronic    147
hiphop         80
             ... 
balkan          1
chamber         1
russian         1
emo             1
adult           1
Name: genre, Length: 106, dtype: int64

Понедельник утро:
6 самых популярных жанров одинаковые.
Дальше в Москве появляется world, а в Питере jazz.

In [219]:
count_by_time('Moscow', 'Monday', '17:00', '21:00', moscow_genres)

genre
pop           377
rock          279
dance         278
electronic    258
hiphop        126
             ... 
reggaeton       1
dub             1
religious       1
audiobook       1
lounge          1
Name: genre, Length: 139, dtype: int64

In [220]:
count_by_time('Saint-Petersburg', 'Monday', '17:00', '21:00', spb_genres)

genre
pop            143
rock           119
electronic      99
dance           80
hiphop          51
              ... 
progressive      1
punjabi          1
european         1
religious        1
christian        1
Name: genre, Length: 89, dtype: int64

Понедельник вечер:
Из отличий можно отметить только то, что в Москве есть ruspop, а в Питере rusrap.

In [221]:
count_by_time('Moscow', 'Friday', '08:00', '12:00', moscow_genres)

genre
pop           781
dance         549
electronic    480
rock          474
hiphop        285
             ... 
orchestral      1
podcasts        1
retro           1
rhythm          1
worldbeat       1
Name: genre, Length: 152, dtype: int64

In [222]:
count_by_time('Saint-Petersburg', 'Friday', '08:00', '12:00', spb_genres)

genre
pop           218
dance         182
rock          162
electronic    147
hiphop         80
             ... 
balkan          1
chamber         1
russian         1
emo             1
adult           1
Name: genre, Length: 106, dtype: int64

Пятница утро:
Из отличий в Питере jazz, а в Москве world.

In [223]:
count_by_time('Moscow', 'Friday', '17:00', '21:00', moscow_genres)

genre
pop           377
rock          279
dance         278
electronic    258
hiphop        126
             ... 
reggaeton       1
dub             1
religious       1
audiobook       1
lounge          1
Name: genre, Length: 139, dtype: int64

In [224]:
count_by_time('Saint-Petersburg', 'Friday', '17:00', '21:00', spb_genres)

genre
pop            143
rock           119
electronic      99
dance           80
hiphop          51
              ... 
progressive      1
punjabi          1
european         1
religious        1
christian        1
Name: genre, Length: 89, dtype: int64

Пятница вечер:
В Питере rusrap, в Москве ruspop.

В итоге можно сказать, что гипотеза подтвердилась лишь отчасти. Разница в предпочтениях наблюдается только утром в том, что в Москве слушают больше world, а в Питере jazz, и вечером только в том, что в Москве ruspop, а в Питере rusrap.



Проверим третью гипотезу. 
Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [225]:
spb_genres.groupby('genre')['user_id'].count().sort_values(ascending=False)

genre
pop           2431
dance         1932
rock          1879
electronic    1737
hiphop         960
              ... 
mexican          1
mandopop         1
leftfield        1
laiko            1
worldbeat        1
Name: user_id, Length: 208, dtype: int64

In [226]:
moscow_genres.groupby('genre')['user_id'].count().sort_values(ascending=False)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2095
               ... 
metalcore         1
marschmusik       1
malaysian         1
lovers            1
ïîï               1
Name: user_id, Length: 262, dtype: int64

Третье гипотеза не подтвердилась. В Москве действительно преобладает поп-музыка, но популярность рэпа в обоих городах одинаковая. 